Muslimov Arthur, Last Checkpoint: 03/11/2020 <br/>
                 Last Checkpoint: 12/05/2021

In [1]:
import numpy as np

Раньше мы создавали массивы, хранящие один тип данных. Как оказалось,        <br/>
если заранее определить типы в массиве, то можно не ограничивать себя        <br/>
этим. Здесь ты познаешь ***структурированные массивы*** (structured arrays)  <br/>
и ***массивы записей*** (record arrays), способные эффективно хранить        <br/>
неоднородные данные в пределах одного блока памяти. Хотя демонстрируемые     <br/>
паттерны удобно для простых операций, они также применими и для структур     <br/>
данных библиотеки Pandas, а имеено DataFrame, который мы обсудим позже.

In [2]:
# Традиционное хранение
x = np.zeros(4, dtype=int)  # подчеркну, dtype=int
x

array([0, 0, 0, 0])

In [3]:
name = ["Alice", "Bob", "Cathy", "Doug"]
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

print("name:  ", name[0])
print("age:   ", age[0])
print("weight:", weight[0])

name:   Alice
age:    25
weight: 55.0


А вот как более грамомотно хранить данные, используя структурированный массив.

In [4]:
data = np.zeros(4, dtype={"names":("name", "age", "weight"),
                          "formats":('U10', 'i4', 'f8')})  # что-то новенькое
print(data.dtype)

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


`'U10'` означает "строка в кодировке Unicode максимальной длины 10",      <br/>
`'i4'` - "4-байтное целое число", а `'f8'` - 8-байтное число с плавающей  <br/>
точкой. Другие варианты типов ты увидишь в следующем подразделе.

In [5]:
data["name"] = name
data["age"] = age
data["weight"] = weight
print(data)

[('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


Такой вариант хранения несколько удобнее. Доступ к данным облегчён.

In [6]:
data["name"]  # выводим имена

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

In [7]:
data[0]  # выводим первую строку данных

('Alice', 25, 55.)

In [8]:
data[-1]["name"]  # имя последней строки

'Doug'

Напоминает csv-формат, не так ли?

Фишки обычных массивов им также доступны. Например, вот булевая маска.

In [9]:
data[data["age"] < 30]["name"]  # имена людей с возрастом менее 30

array(['Alice', 'Doug'], dtype='<U10')

Если ты хочешь сделать фокус покруче, то лучше использовать Pandas,  <br/>
объект DataFrame которого основан на массивах NumPy и имеет          <br/>
массу полезной функциональности по работе с данными.

## Создание структурированных массивов

Есть несколько способов задания типа данных для структурированных массисов.  <br/>
Ты уже видел метод использования словаря.

In [10]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

Кстати, форматы можно задавать и привычными типами Python или NumPy.

In [11]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float64)})  # да, есть и np.str_

dtype([('name', '<U10'), ('age', '<i8'), ('weight', '<f8')])

Составные типы можно задать и без словаря - списком кортежей, например.

In [12]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])  # S10 тоже str, но в Unicode

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

Можно обойтись и без названий типов, отправив просто строку самих типов.

In [13]:
np.dtype('S10, i4, f8')  # типы отправляются строкой и разделяются запятыми

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

Да, строковые коды форматов - немного непревычное явление, но затруднений        <br/>
для восприятия они вызвать не должны. Все они подчиняются простым принципам.     <br/>
Первый символ необязателен - `'<'` или `'>'` - "число с прямым порядком байтов"  <br/>
или "число с обратным порядком байтов" - задают порядок значащих битов.          <br/>
Следующий символ задаёт уже сам тип данных: символ, байтовый тип, целое число,   <br/>
число с плавающей точкой и т.д. Последние символы задают размер в байтах.

**Символ**   | **Описание**                        | **Пример**
:------------|:------------------------------------|:--------------------------------
`'b'`        | Байтовый тип                        | `np.dtype('b')`
`'i'`        | Знаковое целое число                | `np.dtype('i4')` == `np.int32`
`'u'`        | Беззнаковое целое число             | `np.dtype(u1)` == `np.uint8`
`'f'`        | Число с плавающей точкой            | `np.dtype(f8)` == `float`
`'c'`        | Комплексное число с плавающе точкой | `np.dtype(c16)` == `np.complex128`
`'S'`, `'a'` | Строка                              | `np.dtype(a12)`
`'U'`        | Строка в кодировке Unicode          | `np.dtype(U)` == `np.str_`
`'V'`        | Неформатированные данные (тип void) | `np.dtype(V)` == `np.void`

## Более продвинутые типы данных

Можно создавать ещё более продвинутые типы данных, хранящие числа и массивы бок о бок.

In [15]:
tp = np.dtype([('id', 'i4'), ('matrix', 'f8', (3, 3))]) # тритий элемент в кортеже - это форма
X = np.zeros(1, dtype=tp)
print(X[0])  # по сути, можно было вывести и весь X, т.к. он хранит только один элемент
print(X['matrix'][0])  # можно и x[0]['matrix']. Библиотека не запутается

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Мы создали новый тип, хранящий целое число `id` и матрицу float'ов `matrix`,   <br/>
размером 3×3. Почему мы используем массив, а не словари или класс? Потому      <br/>
что `dtype` из NumPy напрямую отражает вид структуры языка C, так что          <br/>
никаких посредников между тобой и данными нет. Эта возможность очень полезна,  <br/>
если ты хочешь написать интерфейс на Python для какой-нибудь C или Fortran     <br/>
библиотеки, использующей структуры как хранилища данных, а таковы почти все.

## Массивы записей: структурированные массивы с дополнительными возможностями

NumPy предоставляет класс `np.recarray`, позволяющий получить доступ полям  <br/>
как к атрибутам, а не только как ключам словаря. Вот так мы делали раньше:

In [16]:
data['age']

array([25, 45, 37, 19], dtype=int32)

> Если же представить наши данные как массив записей, то можно обращаться  <br/>
> к этим данным с помощью чуть более короткого синтаксиса:

In [20]:
data_rec = data.view(np.recarray)  # метод view даёт возможность преобразовать тип данных входного массива
data_rec.age

array([25, 45, 37, 19], dtype=int32)

Недостаток такого подхода в том, что доступ к полям массива записей влечёт  <br/>
дополнительные накладные расходы, даже если ты использует синтаксис словаря.

In [21]:
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

122 ns ± 2.55 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
2.56 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.31 µs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


> Имеет ли смысл жертвовать дополнительным временем ради более  <br/>
> удобного синтаксиса - зависит от вашего приложения.

## Вперёд, к Pandas

> Этот раздел по структурированным массивам и массивам записей не случайно      <br/>
> стоит в конце этой главы, поскольку он удачно подводит нас к теме             <br/>
> следующего рассматриваемого пакета - библиотеки Pandas. В определённых        <br/>
> случаях не помешает знать о существовании обсуждавшихся здесь                 <br/>
> структурированных массивах, ососбенно если вам нужно, чтобы массивы           <br/>
> библиотеки NumPy соответствовали двоичным форматам данных в C, Fortran        <br/>
> или другом языке программирования. Для регулярной работы со                   <br/>
> структурированными данными намного удобнее использовать пакет Pandas, который <br/>
> мы подробно рассмотрим в следующией главе.